<a href="https://colab.research.google.com/github/Bhaanupriyaranjit/DLProject/blob/main/Shakespearellm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [2]:
!pip install torch torchvision matplotlib
import torch
import torch.nn as nn
from torch.nn import functional as F
import urllib.request
import matplotlib.pyplot as plt
print("GPU available:", torch.cuda.is_available())


GPU available: True
